In [1]:
import json
import pandas as pd
import ast
import datetime

In [2]:
def convert_unixtime(utime):
    return (datetime.datetime.fromtimestamp(int(utime)).strftime('%d-%m'))

def convert_country_code(code):
    mapping = {"CO":"Colombia", "BR": "Brazil", "VE": "Venezuela", "PE": "Peru", "UY": "Uruguay", 
               "PY": "Paraguay", "PA": "Panama", "CL": "Chile", "EC":"Ecuador", "AR": "Argentina",
               "CR": "Costa Rica", "GY": "Guyana", "MX": "Mexico", "US": "United States",  
              }
    return mapping[code]

def cut(df, col, values, savedcols= ['Location', 'dau_audience', 'mau_audience']):
    df_slice = {}
    
    for i, v in enumerate(values):
        df_slice[i] = df[df[col] == v][savedcols]
    
    merge = pd.merge(df_slice[0], df_slice[1], on=["Location"], suffixes=("_%s" % (values[0]), "_%s" % (values[1])))
    for i in range(2, len(values)):
        #print("Adding suffix for:" + values[i])
        merge = pd.merge(merge, df_slice[i], on=["Location"], suffixes=("", "_%s" % (values[i])))
        
    if len(values) > 2:
        # Fix information for the second group.
        merge["audience_%s" % (values[2])] = merge["audience"]
        del merge["audience"]
    return merge   

def copy_rename(df, oldname, newname):
    return df.rename(columns={oldname: newname}).copy() 

def get_slice(dfin, col, values, frequency="mau"):
    dfout = copy_rename(dfin, '%s_audience' % (frequency), "audience")
    dfout = cut(dfout, col, values, savedcols=['Location', 'audience'])
    dfout["Frequency"] = "Daily" if frequency == "dau" else "Monthly"
    return dfout

def calculate_percentages(df, cols, prefix):
    totals = df.sum(axis=1)
    for col in cols:
        df[prefix + "audience_" + col] = df["audience_" + col] * 100. / totals
        df[prefix + "audience_" + col] = df["audience_" + col] * 100. / totals

In [88]:
#infile = "collections/colombia/dataframe_collected_finished_1526497778.csv.gz" # From 16th May
#infile = "collections/colombia/dataframe_collected_finished_1526595686.csv.gz" # From 18th May
#infile = "collections/colombia/dataframe_collected_finished_1526754549.csv.gz" # From 19th May
#infile = "collections/colombia/dataframe_collected_finished_1527055300.csv.gz" # From 23th May   
#infile = "collections/colombia/dataframe_collected_finished_1527086071.csv.gz" # From 23th May # Needs to fix duplicate for Santander, Colombia in all datasets before this one
#infile = "collections/colombia/dataframe_collected_finished_1527152527.csv.gz" # From 24th May
#infile = "collections/colombia/dataframe_collected_finished_1527162372.csv.gz" # From 24th May
#infile = "collections/colombia/dataframe_collected_finished_1527262566.csv.gz" # From 25th May
#infile = "collections/colombia/dataframe_collected_finished_1527336923.csv.gz" # From 26th May
#infile = "collections/colombia/dataframe_collected_finished_1527421314.csv.gz" # From 27th May
#infile = "collections/colombia/dataframe_collected_finished_1527620434.csv.gz" # From 28th May
#infile = "collections/colombia/dataframe_collected_finished_1527655611.csv.gz" # From 29th May
#infile = "collections/colombia/dataframe_collected_finished_1527670079.csv.gz" # From 30th May
#infile = "collections/colombia/dataframe_collected_finished_1527714408.csv.gz" # From 31th May
#infile = "collections/colombia/dataframe_collected_finished_1527854965.csv.gz" # From 1st June
#infile = "collections/colombia/dataframe_collected_finished_1528052984.csv.gz" # From 3rd June
#infile = "collections/colombia/dataframe_collected_finished_1528190439.csv.gz" # From 5th June
#infile = "collections/colombia/dataframe_collected_finished_1528297332.csv.gz" # From 6th June
#infile = "collections/colombia/dataframe_collected_finished_1528410926.csv.gz" # From 8th June
#infile = "collections/colombia/dataframe_collected_finished_1528531672.csv.gz" # From 9th June
#infile = "collections/colombia/dataframe_collected_finished_1528635898.csv.gz" # From 10th June
#infile = "collections/colombia/dataframe_collected_finished_1528793179.csv.gz" # From 12th June
#infile = "collections/colombia/dataframe_collected_finished_1528925661.csv.gz" # From 14th June
#infile = "collections/colombia/dataframe_collected_finished_1529055941.csv.gz" # From 15th June
#infile = "collections/colombia/dataframe_collected_finished_1529181638.csv.gz" # From 16th June
#infile = "collections/colombia/dataframe_collected_finished_1529319680.csv.gz" # From 18th June
##### Missing:
#infile = "collections/colombia/dataframe_collected_finished_1529483108.csv.gz" # From 20th June
#infile = "collections/colombia/dataframe_collected_finished_1529789939.csv.gz" # From 24th June
#infile = "collections/colombia/dataframe_collected_finished_1529923559.csv.gz" # From 25th June
#infile = "collections/colombia/dataframe_collected_finished_1530043687.csv.gz" # From 26th June
#infile = "collections/colombia/dataframe_collected_finished_1530166519.csv.gz" # From 28th June
infile = "collections/colombia/dataframe_collected_finished_1530302641.csv.gz" # From 29th June

df = pd.read_csv(infile)
df["CollectionDay"] = df["timestamp"].apply(lambda x: convert_unixtime(x))
print("Collection made in %s" % (df["CollectionDay"].head()[0]))

/home/palotti/p3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Collection made in 29-06


In [89]:
def extract_relationship(d):
    if 1 in d:
        return "single"
    elif 2 in d:
        return "dating"
    elif 3 in d:
        return "married"
    else:
        return None

def extract_education(d):
    if d == [3, 7, 8, 9, 11]:
        return "graduated"
    elif d == [1, 12, 13]:
        return "no_degree"
    elif d == [2, 4, 5, 6, 10]:
        return "high_school"
    elif d == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
        return "all"
    else:
        return None

def extract_group(d):
    id = d[0]["id"]
    if id == 6026404871583:
        return "Expats (Venezuela)"
    elif id == 6015559470583:
        return "Ex-pats (All)"
    else:
        return "All"

def agebuckets(minage, maxage):
    if minage == 13 and (maxage is None or np.isnan(maxage)):
        return "all"
    elif minage == 13 and maxage == 18:
        return "adolecent"
    elif minage == 19 and maxage == 25:
        return "young_adult"
    elif minage == 26 and maxage == 40:
        return "adult"
    elif minage == 41 and maxage == 65:
        return "middle_age"
    elif minage == 65 and (maxage is None or np.isnan(maxage)):
        return "elder"
    return "undefined"
    
def expand(row):
    place = None
    loc_dimension = None
    if "regions" in row["geo_locations"]:
        place = "%s, %s" % (row["geo_locations"]["regions"][0]["name"], convert_country_code(row["geo_locations"]["regions"][0]["country_code"]))
        loc_dimension = "State"
    elif "countries" in row["geo_locations"]:
        place = convert_country_code(row["geo_locations"]["countries"][0])
        loc_dimension = "Country"
    elif "cities" in row["geo_locations"]:
        place = "%s, %s, %s" % (row["geo_locations"]["cities"][0]["name"], row["geo_locations"]["cities"][0]["region"], convert_country_code(row["geo_locations"]["cities"][0]["country"]))
        loc_dimension = "City"
    
    loctype = "_".join(row["geo_locations"]["location_types"])

    relationship, education, group = None, None, None
    for dimension in row["flexible_spec"]:
        if "relationship_statuses" in dimension:
            relationship = extract_relationship(dimension["relationship_statuses"])
        elif "education_statuses" in dimension:
            education = extract_education(dimension["education_statuses"])
        elif "behaviors" in dimension:
            group = extract_group(dimension["behaviors"])
    
    gender = row["genders"][0]
    gender = "both" if gender == 0 else "man" if gender == 1 else "woman"
    
    return row["age_min"], row["age_max"], place, loc_dimension, loctype, gender, relationship, education, group

def get_item(x):
    if not x:
        return None
    return x["name"]

df[["MinAge","MaxAge","Location","LocationHierarchy","LocationType","Gender","Relationship","Education","Group"]] = df["targeting"].apply(lambda x : expand(ast.literal_eval(x))).apply(pd.Series)

if "citizenship" in df:
    df["Group"] = df["citizenship"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Updating Group information...")
    
if "access_device" in df:
    df["Device"] = df["access_device"].fillna("[]").apply(lambda x : get_item(ast.literal_eval(x)))
    print("Adding information regarding devices...")
else:
    df["Device"] = None
    
df["AgeBucket"] = df[["MinAge","MaxAge"]].apply(lambda x: agebuckets(x["MinAge"], x["MaxAge"]), axis=1)

print("Removing redundant cols")
for col in ["Unnamed: 0", "all_fields", "targeting",'behavior', 'citizenship', "mock_response", "access_device", 
            "ages_ranges", "household_composition", 'interests', 'family_statuses', 'genders', 'geo_locations', 'languages',
            'name', 'relationship_statuses', 'response', 'scholarities', 'timestamp', 'publisher_platforms',]:
    if col in df.keys():
        del df[col]

# Gave up using travel_in   
df = df[df["LocationType"] != "travel_in"]

print("All Done!")
# Brief description: 
# -----------------
# Gender: 0 Both, 1 Man, 2 Woman
# Age: 13-Null, 13-18, 19-25, 26-40, 41-65, 65+
# LocationType: home_recent, home, recent


Updating Group information...
Adding information regarding devices...
Removing redundant cols
All Done!


In [90]:
dfgender = {}
dfcut = df[(df["Gender"].apply(lambda x : x in ["man","woman"])) & (df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent")].copy()
dfgender["mau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="mau")
dfgender["dau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="dau")
calculate_percentages(dfgender["mau"], ["man","woman"], "%ven_")
calculate_percentages(dfgender["dau"], ["man","woman"], "%ven_")

In [105]:
df_gender_locals = {}
dfcut = df[(df["Gender"].apply(lambda x : x in ["man","woman"])) & (df["Education"].isnull()) & (df["Device"].isnull()) 
           & (df["AgeBucket"] == "all") & (df["Group"].isnull()) & (df["LocationType"] == "home_recent") ].copy()

df_gender_locals["mau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="mau") 
df_gender_locals["dau"] = get_slice(dfcut, "Gender", ["man","woman"], frequency="dau")

calculate_percentages(df_gender_locals["mau"], ["man","woman"], "%locals_")
calculate_percentages(df_gender_locals["dau"], ["man","woman"], "%locals_")

for col in ["man","woman"]:
    del df_gender_locals["mau"]["audience_" + col]
    del df_gender_locals["dau"]["audience_" + col]


In [91]:
dfage = {}
dfcut = df[(df["Education"].isnull()) & (df["Device"].isnull()) &  (df["AgeBucket"].apply(lambda x : x in ["adolecent","young_adult","adult","middle_age", "elder"])) &
    (df["Group"] == "Expats (Venezuela)") & (df["Gender"] == "both") & (df["LocationType"] == "home_recent") ].copy()
dfage["mau"] = get_slice(dfcut, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="mau") 
dfage["dau"] = get_slice(dfcut, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="dau")

calculate_percentages(dfage["mau"], ["adolecent","young_adult","adult","middle_age", "elder"], "%ven_")
calculate_percentages(dfage["dau"], ["adolecent","young_adult","adult","middle_age", "elder"], "%ven_")


In [92]:
df_age_pyramid = {}
dfcut_man = df[(df["AgeBucket"].apply(lambda x : x in ["adolecent","young_adult","adult","middle_age", "elder"])) & (df["Gender"] == "man") &
           (df["Education"].isnull()) & (df["Device"].isnull()) &  (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent") ]
df_age_pyramid["mau_man"] = get_slice(dfcut_man, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="mau")
df_age_pyramid["dau_man"] = get_slice(dfcut_man, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="dau")

dfcut_woman = df[(df["AgeBucket"].apply(lambda x : x in ["adolecent","young_adult","adult","middle_age", "elder"])) & (df["Gender"] == "woman") &
           (df["Education"].isnull()) & (df["Device"].isnull()) &  (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent") ]
df_age_pyramid["mau_woman"] = get_slice(dfcut_woman, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="mau")
df_age_pyramid["dau_woman"] = get_slice(dfcut_woman, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="dau")

df_age_pyramid["dau"] = pd.merge(df_age_pyramid["dau_man"],df_age_pyramid["dau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))
df_age_pyramid["mau"] = pd.merge(df_age_pyramid["mau_man"],df_age_pyramid["mau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))


In [93]:
df_age_locals = {}
dfcut = df[(df["Education"].isnull()) & (df["Device"].isnull()) &  (df["AgeBucket"].apply(lambda x : x in ["adolecent","young_adult","adult","middle_age", "elder"])) &
    (df["Group"].isnull()) & (df["Gender"] == "both") & (df["LocationType"] == "home_recent") ].copy()
df_age_locals["mau"] = get_slice(dfcut, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="mau") 
df_age_locals["dau"] = get_slice(dfcut, "AgeBucket", ["adolecent","young_adult","adult","middle_age", "elder"], frequency="dau")

calculate_percentages(df_age_locals["mau"], ["adolecent","young_adult","adult","middle_age", "elder"], "%locals_")
calculate_percentages(df_age_locals["dau"], ["adolecent","young_adult","adult","middle_age", "elder"], "%locals_")

for col in ["adolecent","young_adult","adult","middle_age", "elder"]:
    del df_age_locals["mau"]["audience_" + col]
    del df_age_locals["dau"]["audience_" + col]


In [94]:
dfloctype = {}
dfcut = df[(df["LocationType"].apply(lambda x : x in ["home","recent","travel_in","home_recent"])) & (df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"] == "Expats (Venezuela)") & (df["Gender"] == "both") ]
dfloctype["mau"] = get_slice(dfcut, "LocationType", ["home","recent"], frequency="mau")
dfloctype["dau"] = get_slice(dfcut, "LocationType", ["home","recent"], frequency="dau")

calculate_percentages(dfloctype["mau"], ["home","recent"], "%ven_")
calculate_percentages(dfloctype["dau"], ["home","recent"], "%ven_")

In [95]:
df_loctype_pyramid = {}
#dfcut_man = df[(~df["Education"].isnull()) & (df["Gender"] == "man") & (df["AgeBucket"] == "all") & (df["Device"].isnull()) &  (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent") ]
dfcut_man = df[(df["LocationType"].apply(lambda x : x in ["home","recent"])) & (df["Device"].isnull()) & (df["Education"].isnull()) & (df["Gender"] == "man") & (df["AgeBucket"] == "all") &  (df["Group"] == "Expats (Venezuela)") ]
df_loctype_pyramid["mau_man"] = get_slice(dfcut_man, "LocationType", ["home","recent"], frequency="mau")
df_loctype_pyramid["dau_man"] = get_slice(dfcut_man, "LocationType", ["home","recent"], frequency="dau")

dfcut_woman = df[(df["LocationType"].apply(lambda x : x in ["home","recent"])) & (df["Device"].isnull()) & (df["Education"].isnull()) & (df["Gender"] == "woman") & (df["AgeBucket"] == "all") &  (df["Group"] == "Expats (Venezuela)") ]
df_loctype_pyramid["mau_woman"] = get_slice(dfcut_woman, "LocationType", ["home","recent"], frequency="mau")
df_loctype_pyramid["dau_woman"] = get_slice(dfcut_woman, "LocationType", ["home","recent"], frequency="dau")

df_loctype_pyramid["dau"] = pd.merge(df_loctype_pyramid["dau_man"],df_loctype_pyramid["dau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))
df_loctype_pyramid["mau"] = pd.merge(df_loctype_pyramid["mau_man"],df_loctype_pyramid["mau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))

In [96]:
df_loctype_locals = {}
dfcut = df[(df["LocationType"].apply(lambda x : x in ["home","recent"])) & (df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") &
    (df["Group"].isnull()) & (df["Gender"] == "both") ].copy()
df_loctype_locals["mau"] = get_slice(dfcut, "LocationType", ["home","recent"], frequency="mau") 
df_loctype_locals["dau"] = get_slice(dfcut, "LocationType", ["home","recent"], frequency="dau")

calculate_percentages(df_loctype_locals["mau"], ["home","recent"], "%locals_")
calculate_percentages(df_loctype_locals["dau"], ["home","recent"], "%locals_")

for col in ["home","recent"]:
    del df_loctype_locals["mau"]["audience_" + col]
    del df_loctype_locals["dau"]["audience_" + col]


In [97]:
dfdevice = {}
dfcut = df[(df["Education"].isnull()) & (~df["Device"].isnull()) & (df["AgeBucket"] == "all") &  (df["Group"] == "Expats (Venezuela)") & (df["Gender"] == "both") & (df["LocationType"] == "home_recent") ]
dfdevice["mau"] = get_slice(dfcut, "Device", ["iOS","Android","Other"], frequency="mau")
dfdevice["dau"] = get_slice(dfcut, "Device", ["iOS","Android","Other"], frequency="dau")

calculate_percentages(dfdevice["mau"], ["iOS","Android","Other"], "%ven_")
calculate_percentages(dfdevice["dau"], ["iOS","Android","Other"], "%ven_")


In [98]:
df_device_pyramid = {}

dfcut_man = df[(~df["Device"].isnull()) & (df["Education"].isnull()) & (df["Gender"] == "man") & (df["AgeBucket"] == "all") & (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent") ]
df_device_pyramid["mau_man"] = get_slice(dfcut_man, "Device", ["iOS","Android","Other"], frequency="mau")
df_device_pyramid["dau_man"] = get_slice(dfcut_man, "Device", ["iOS","Android","Other"], frequency="dau")

dfcut_woman = df[(~df["Device"].isnull()) & (df["Education"].isnull()) & (df["Gender"] == "woman") & (df["AgeBucket"] == "all") & (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent") ]

df_device_pyramid["mau_woman"] = get_slice(dfcut_woman, "Device", ["iOS","Android","Other"], frequency="mau")
df_device_pyramid["dau_woman"] = get_slice(dfcut_woman, "Device", ["iOS","Android","Other"], frequency="dau")

df_device_pyramid["dau"] = pd.merge(df_device_pyramid["dau_man"],df_device_pyramid["dau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))
df_device_pyramid["mau"] = pd.merge(df_device_pyramid["mau_man"],df_device_pyramid["mau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))

In [99]:
df_device_locals = {}
dfcut = df[(~df["Device"].isnull()) & (df["Education"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"].isnull()) & (df["Gender"] == "both") & (df["LocationType"] == "home_recent") ].copy()
df_device_locals["mau"] = get_slice(dfcut, "Device", ["iOS","Android","Other"], frequency="mau") 
df_device_locals["dau"] = get_slice(dfcut, "Device", ["iOS","Android","Other"], frequency="dau")

calculate_percentages(df_device_locals["mau"], ["iOS","Android","Other"], "%locals_")
calculate_percentages(df_device_locals["dau"], ["iOS","Android","Other"], "%locals_")

for col in ["iOS","Android","Other"]:
    del df_device_locals["mau"]["audience_" + col]
    del df_device_locals["dau"]["audience_" + col]


In [100]:
dfedu = {}
dfcut = df[(~df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") &  (df["Group"] == "Expats (Venezuela)") & (df["Gender"] == "both") & (df["LocationType"] == "home_recent") ]
dfedu["mau"] = get_slice(dfcut, "Education", ["graduated", "high_school"], frequency="mau")
dfedu["dau"] = get_slice(dfcut, "Education", ["graduated", "high_school"], frequency="dau")

calculate_percentages(dfedu["mau"], ["graduated", "high_school"], "%ven_")
calculate_percentages(dfedu["dau"], ["graduated", "high_school"], "%ven_")


In [101]:
df_edu_pyramid = {}
dfcut_man = df[(~df["Education"].isnull()) & (df["Gender"] == "man") & (df["AgeBucket"] == "all") & (df["Device"].isnull()) &  (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent") ]
df_edu_pyramid["mau_man"] = get_slice(dfcut_man, "Education", ["graduated", "high_school"], frequency="mau")
df_edu_pyramid["dau_man"] = get_slice(dfcut_man,  "Education", ["graduated", "high_school"], frequency="dau")

dfcut_woman = df[(~df["Education"].isnull()) & (df["Gender"] == "woman") & (df["AgeBucket"] == "all") & (df["Device"].isnull()) &  (df["Group"] == "Expats (Venezuela)") & (df["LocationType"] == "home_recent") ]
df_edu_pyramid["mau_woman"] = get_slice(dfcut_woman, "Education", ["graduated", "high_school"], frequency="mau")
df_edu_pyramid["dau_woman"] = get_slice(dfcut_woman, "Education", ["graduated", "high_school"], frequency="dau")

df_edu_pyramid["dau"] = pd.merge(df_edu_pyramid["dau_man"],df_edu_pyramid["dau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))
df_edu_pyramid["mau"] = pd.merge(df_edu_pyramid["mau_man"],df_edu_pyramid["mau_woman"], on=["Location","Frequency"], suffixes=("_man","_woman"))


In [102]:
df_edu_locals = {}
dfcut = df[~(df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") &
    (df["Group"].isnull()) & (df["Gender"] == "both") & (df["LocationType"] == "home_recent") ].copy()
df_edu_locals["mau"] = get_slice(dfcut, "Education", ["graduated", "high_school"], frequency="mau") 
df_edu_locals["dau"] = get_slice(dfcut, "Education", ["graduated", "high_school"], frequency="dau")

calculate_percentages(df_edu_locals["mau"], ["graduated", "high_school"], "%locals_")
calculate_percentages(df_edu_locals["dau"], ["graduated", "high_school"], "%locals_")

for col in ["graduated", "high_school"]:
    del df_edu_locals["mau"]["audience_" + col]
    del df_edu_locals["dau"]["audience_" + col]


In [103]:
for dfopt in [dfgender, dfage, dfedu, dfdevice, dfloctype, 
              df_age_pyramid, df_edu_pyramid, df_device_pyramid, df_loctype_pyramid,
              df_age_locals, df_edu_locals, df_device_locals,  df_loctype_locals]:
    dfopt["dau"] = dfopt["dau"].drop_duplicates()
    dfopt["mau"] = dfopt["mau"].drop_duplicates()

In [112]:
merged = {}

merged["mau"] = pd.merge(dfgender["mau"], dfloctype["mau"]).merge(dfage["mau"]).merge(df_gender_locals["mau"])
merged["mau"] = merged["mau"].merge(df_age_locals["mau"]).merge(df_age_pyramid["mau"]).merge(df_loctype_locals["mau"]).merge(df_loctype_pyramid["mau"])

merged["dau"] = pd.merge(dfgender["dau"], dfloctype["dau"]).merge(dfage["dau"]).merge(df_gender_locals["dau"])
merged["dau"] = merged["dau"].merge(df_age_locals["dau"]).merge(df_age_pyramid["dau"]).merge(df_loctype_locals["dau"]).merge(df_loctype_pyramid["dau"])

if not dfdevice["mau"].empty:
    merged["mau"] = merged["mau"].merge(dfdevice["mau"]).merge(df_device_pyramid["mau"]).merge(df_device_locals["mau"])
    merged["dau"] = merged["dau"].merge(dfdevice["dau"]).merge(df_device_pyramid["dau"]).merge(df_device_locals["dau"])
    
if not dfedu["mau"].empty:
    merged["mau"] = merged["mau"].merge(dfedu["mau"]).merge(df_edu_pyramid["mau"]).merge(df_edu_locals["mau"])
    merged["dau"] = merged["dau"].merge(dfedu["dau"]).merge(df_edu_pyramid["dau"]).merge(df_edu_locals["dau"])
       
concated = pd.concat([merged["mau"], merged["dau"]])

venezuelans = df[(df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"] == "Expats (Venezuela)") & (df["Gender"] == "both") & (df["LocationType"] == "home_recent")]
allmigrants = df[(df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"] == "Ex-pats (All)") & (df["Gender"] == "both") & (df["LocationType"] == "home_recent")]
allpopulation = df[(df["Education"].isnull()) & (df["Device"].isnull()) & (df["AgeBucket"] == "all") & (df["Group"].isnull()) & (df["Gender"] == "both") & (df["LocationType"] == "home_recent")]

dffinal = pd.merge(venezuelans[["dau_audience","mau_audience","Location","LocationHierarchy"]], concated)
dffinal["venezuelans"] = dffinal["mau_audience"].where( dffinal["Frequency"]=="Monthly", dffinal["dau_audience"]) 
del dffinal["mau_audience"]
del dffinal["dau_audience"]

dffinal = pd.merge(allmigrants[["dau_audience","mau_audience","Location","LocationHierarchy"]], dffinal)
dffinal["allmigrants"] = dffinal["mau_audience"].where( dffinal["Frequency"]=="Monthly", dffinal["dau_audience"]) 
del dffinal["mau_audience"]
del dffinal["dau_audience"]

dffinal = pd.merge(allpopulation[["dau_audience","mau_audience","Location","LocationHierarchy"]], dffinal)
dffinal["allpopulation"] = dffinal["mau_audience"].where( dffinal["Frequency"]=="Monthly", dffinal["dau_audience"]) 
del dffinal["mau_audience"]
del dffinal["dau_audience"]

dffinal["ven/migrants"] = dffinal["venezuelans"] / dffinal["allmigrants"]
dffinal["ven/pop"] = dffinal["venezuelans"] / dffinal["allpopulation"]

dffinal["100ven/migrants"] = 100. * dffinal["ven/migrants"]
dffinal["100ven/pop"] = 100. * dffinal["ven/pop"]

for trans_col in ["ven/migrants", "ven/pop"]:
    dffinal[trans_col] = dffinal[trans_col].apply(lambda x: "%.4f" % x)
for trans_col in ["100ven/migrants", "100ven/pop"]:
    dffinal[trans_col] = dffinal[trans_col].apply(lambda x: "%.2f" % x)

dffinal["not_enough_data"] = False
dffinal.loc[dffinal["venezuelans"] <= 1000, "not_enough_data"] = True 

for col in ["home","recent"] + ["man","woman"] + ["adolecent","young_adult","adult","middle_age", "elder"] + ["graduated", "high_school"] +  ["iOS","Android","Other"]:
    dffinal["%diff_audience_" + col] = dffinal["%ven_audience_" + col] - dffinal["%locals_audience_" + col]

for col in dffinal.keys():
    if col.startswith("%"):
        print(col)
        dffinal[col] = dffinal[col].round(2)

dffinal.to_csv("merged_06_29.csv", index=False)

%ven_audience_man
%ven_audience_woman
%ven_audience_home
%ven_audience_recent
%ven_audience_adolecent
%ven_audience_young_adult
%ven_audience_adult
%ven_audience_middle_age
%ven_audience_elder
%locals_audience_man
%locals_audience_woman
%locals_audience_adolecent
%locals_audience_young_adult
%locals_audience_adult
%locals_audience_middle_age
%locals_audience_elder
%locals_audience_home
%locals_audience_recent
%ven_audience_iOS
%ven_audience_Android
%ven_audience_Other
%locals_audience_iOS
%locals_audience_Android
%locals_audience_Other
%ven_audience_graduated
%ven_audience_high_school
%locals_audience_graduated
%locals_audience_high_school
%diff_audience_home
%diff_audience_recent
%diff_audience_man
%diff_audience_woman
%diff_audience_adolecent
%diff_audience_young_adult
%diff_audience_adult
%diff_audience_middle_age
%diff_audience_elder
%diff_audience_graduated
%diff_audience_high_school
%diff_audience_iOS
%diff_audience_Android
%diff_audience_Other
